In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
# import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import utils.utils as util

import numpy as np
import os, time, sys
import argparse

import utils.pg_utils as q
import model as m

In [2]:
def load_cifar10():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform_train = transforms.Compose([
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomCrop(32, 4),
                    transforms.ToTensor(),
                    normalize
        ])
    transform_test = transforms.Compose([
                    transforms.ToTensor(),
                    normalize
        ])

    # pin_memory=True makes transfering data from host to GPU faster
    trainset = torchvision.datasets.CIFAR10(root='/tmp/cifar10_data', train=True,
                                            download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                              shuffle=True, num_workers=2, pin_memory=True)

    testset = torchvision.datasets.CIFAR10(root='/tmp/cifar10_data', train=False,
                                           download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                             shuffle=True, num_workers=2, pin_memory=True)

    classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    return trainloader, testloader, classes

In [3]:
def generate_model(model_arch):
    if model_arch == 'resnet-20':
        if pg:
            import model.pg_cifar10_resnet as m
            kwargs = {'wbits':wbits, 'abits':abits, \
                      'pred_bits':pbits, 'sparse_bp':sparse_bp, \
                      'pact':pact}
            return m.resnet20(**kwargs)
        else:
            import model.quantized_cifar10_resnet as m
            kwargs = {'wbits':wbits, 'abits':abits, 'pact':pact}
            return m.resnet20(**kwargs)
    else:
        raise NotImplementedError("Model architecture is not supported.")

In [4]:
from time import perf_counter
import matplotlib.pyplot as plt
from livelossplot import PlotLosses

def train_model(trainloader, testloader, net, device):
    if torch.cuda.device_count() > 1:
        # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
        print("Activate multi GPU support.")
        net = nn.DataParallel(net)
    net.to(device)
    # define the loss function
    criterion = (nn.CrossEntropyLoss().cuda() 
                if torch.cuda.is_available() else nn.CrossEntropyLoss())
    # Scale the lr linearly with the batch size. 
    # Should be 0.1 when batch_size=128
    initial_lr = 0.1 * batch_size / 128
    # initialize the optimizer
    optimizer = optim.SGD(net.parameters(), 
                          lr=initial_lr, 
                          momentum=0.9,
                          weight_decay=_WEIGHT_DECAY)
    # multiply the lr by 0.1 at 100, 150, and 200 epochs
    div = num_epoch // 4
    lr_decay_milestones = [div*2, div*3]
    scheduler = optim.lr_scheduler.MultiStepLR(
                        optimizer, 
                        milestones=lr_decay_milestones, 
                        gamma=0.1,
                        last_epoch=_LAST_EPOCH)
    
    # some bookkeeping
    since = perf_counter()
    liveloss = PlotLosses()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best = 0
    
    loss_list = []
    accuracy_list = []
    val_loss_list = []
    val_acc_list = []
    max_val_acc = 0

    for epoch in range(num_epoch): # loop over the dataset multiple times

        # set printing functions
        batch_time = util.AverageMeter('Time/batch', ':.3f')
        losses = util.AverageMeter('Loss', ':6.2f')
        top1 = util.AverageMeter('Acc', ':6.2f')
        progress = util.ProgressMeter(
                        len(trainloader),
                        [losses, top1, batch_time],
                        prefix="Epoch: [{}]".format(epoch+1)
                        )

        # switch the model to the training mode
        net.train()

        print('current learning rate = {}'.format(optimizer.param_groups[0]['lr']))
        
        # each epoch
        running_loss = 0.0
        running_corrects = 0
        
        end = time.time()
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            for name, param in net.named_parameters():
                if 'threshold' in name:
                    loss += sigma * torch.norm(param-gtarget)
            loss.backward()
            optimizer.step()

            # measure accuracy and record loss
            _, batch_predicted = torch.max(outputs.data, 1)
            batch_accu = 100.0 * (batch_predicted == labels).sum().item() / labels.size(0)
            losses.update(loss.item(), labels.size(0))
            top1.update(batch_accu, labels.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 50 == 49:    
                # print statistics every 100 mini-batches each epoch
                progress.display(i) # i = batch id in the epoch

        # update the learning rate
        scheduler.step()

        # print test accuracy every few epochs
        if epoch % 10 == 9:
            print('epoch {}'.format(epoch+1))
            test_accu(testloader, net, device)

    # save the model if required
    if save:
        print("Saving the trained model.")
        util.save_models(net.state_dict(), save_folder, suffix=_ARCH)

    time_elapsed = perf_counter() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Validation Accuracy: {}, Epoch: {}'.format(best_acc, best))
    return (loss_list, accuracy_list, val_loss_list, val_acc_list)

In [5]:
def test_accu(testloader, net, device):
    net.to(device)
    cnt_out = np.zeros(9) # this 9 is hardcoded for ResNet-20
    cnt_high = np.zeros(9) # this 9 is hardcoded for ResNet-20
    num_out = []
    num_high = []
    def _report_sparsity(m):
        classname = m.__class__.__name__
        if isinstance(m, q.PGConv2d):
            num_out.append(m.num_out)
            num_high.append(m.num_high)

    correct = 0
    total = 0
    # switch the model to the evaluation mode
    net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            """ calculate statistics per PG layer """
            if pg:
                net.apply(_report_sparsity)
                cnt_out += np.array(num_out)
                cnt_high += np.array(num_high)
                num_out = []
                num_high = []

    print('Accuracy of the network on the 10000 test images: %.1f %%' % (
        100 * correct / total))
    if pg:
        print('Sparsity of the update phase: %.1f %%' % (100-np.sum(cnt_high)*1.0/np.sum(cnt_out)*100))

In [6]:
def per_class_test_accu(testloader, classes, net, device):
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(10):
        print('Accuracy of %5s : %.1f %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

In [7]:
batch_size = 32
num_epoch = 200
_LAST_EPOCH = -1 #last_epoch arg is useful for restart
_WEIGHT_DECAY = 1e-4
_ARCH = "resnet-20"
this_file_path = os.path.dirname(os.path.abspath('.'))
save_folder = os.path.join(this_file_path, 'save_CIFAR10_model')
print('Save at', save_folder)

Save at /home/aperture/Git/save_CIFAR10_model


In [8]:
path = None
save = True
test = False
wbits = 8
abits = 3
pact = True
pbits = 2
gtarget = 1
sparse_bp = True
pg = True
sigma = 0.001

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Available GPUs: {}".format(torch.cuda.device_count()))

print("Create {} model.".format(_ARCH))
net = generate_model(_ARCH)
# print(net)

if path:
    print("@ Load trained model from {}.".format(path))
    net.load_state_dict(torch.load(path))

print("Loading the data.")
trainloader, testloader, classes = load_cifar10()
if test:
    print("Mode: Test only.")
    test_accu(testloader, net, device)
else:
    print("Start training.")
    train_model(trainloader, testloader, net, device)
    test_accu(testloader, net, device)
    per_class_test_accu(testloader, classes, net, device)

Available GPUs: 1
Create resnet-20 model.
Loading the data.
Files already downloaded and verified
Files already downloaded and verified
Start training.
current learning rate = 0.025
Epoch: [1][  49/1563]	Loss   2.56	Acc  15.62	Time/batch 0.060
Epoch: [1][  99/1563]	Loss   2.36	Acc  19.78	Time/batch 0.057
Epoch: [1][ 149/1563]	Loss   2.24	Acc  22.50	Time/batch 0.057
Epoch: [1][ 199/1563]	Loss   2.16	Acc  24.72	Time/batch 0.057
Epoch: [1][ 249/1563]	Loss   2.10	Acc  26.26	Time/batch 0.056
Epoch: [1][ 299/1563]	Loss   2.05	Acc  27.35	Time/batch 0.056
Epoch: [1][ 349/1563]	Loss   2.02	Acc  28.48	Time/batch 0.056
Epoch: [1][ 399/1563]	Loss   1.99	Acc  29.49	Time/batch 0.056
Epoch: [1][ 449/1563]	Loss   1.96	Acc  30.34	Time/batch 0.056
Epoch: [1][ 499/1563]	Loss   1.93	Acc  31.21	Time/batch 0.056
Epoch: [1][ 549/1563]	Loss   1.91	Acc  31.82	Time/batch 0.056
Epoch: [1][ 599/1563]	Loss   1.89	Acc  32.69	Time/batch 0.056
Epoch: [1][ 649/1563]	Loss   1.86	Acc  33.60	Time/batch 0.056
Epoch: [1][ 

Epoch: [5][ 249/1563]	Loss   0.75	Acc  75.33	Time/batch 0.056
Epoch: [5][ 299/1563]	Loss   0.74	Acc  75.92	Time/batch 0.056
Epoch: [5][ 349/1563]	Loss   0.74	Acc  75.84	Time/batch 0.056
Epoch: [5][ 399/1563]	Loss   0.73	Acc  76.03	Time/batch 0.056
Epoch: [5][ 449/1563]	Loss   0.74	Acc  75.97	Time/batch 0.056
Epoch: [5][ 499/1563]	Loss   0.74	Acc  75.96	Time/batch 0.056
Epoch: [5][ 549/1563]	Loss   0.73	Acc  75.98	Time/batch 0.056
Epoch: [5][ 599/1563]	Loss   0.74	Acc  75.98	Time/batch 0.056
Epoch: [5][ 649/1563]	Loss   0.74	Acc  75.88	Time/batch 0.056
Epoch: [5][ 699/1563]	Loss   0.74	Acc  75.97	Time/batch 0.056
Epoch: [5][ 749/1563]	Loss   0.74	Acc  75.96	Time/batch 0.056
Epoch: [5][ 799/1563]	Loss   0.73	Acc  76.05	Time/batch 0.056
Epoch: [5][ 849/1563]	Loss   0.73	Acc  76.10	Time/batch 0.056
Epoch: [5][ 899/1563]	Loss   0.73	Acc  76.12	Time/batch 0.056
Epoch: [5][ 949/1563]	Loss   0.73	Acc  76.21	Time/batch 0.056
Epoch: [5][ 999/1563]	Loss   0.73	Acc  76.21	Time/batch 0.056
Epoch: [

Epoch: [9][ 599/1563]	Loss   0.57	Acc  81.38	Time/batch 0.057
Epoch: [9][ 649/1563]	Loss   0.57	Acc  81.23	Time/batch 0.057
Epoch: [9][ 699/1563]	Loss   0.57	Acc  81.13	Time/batch 0.057
Epoch: [9][ 749/1563]	Loss   0.57	Acc  81.22	Time/batch 0.057
Epoch: [9][ 799/1563]	Loss   0.57	Acc  81.32	Time/batch 0.057
Epoch: [9][ 849/1563]	Loss   0.57	Acc  81.26	Time/batch 0.057
Epoch: [9][ 899/1563]	Loss   0.57	Acc  81.26	Time/batch 0.057
Epoch: [9][ 949/1563]	Loss   0.57	Acc  81.41	Time/batch 0.057
Epoch: [9][ 999/1563]	Loss   0.57	Acc  81.43	Time/batch 0.057
Epoch: [9][1049/1563]	Loss   0.57	Acc  81.39	Time/batch 0.057
Epoch: [9][1099/1563]	Loss   0.57	Acc  81.44	Time/batch 0.057
Epoch: [9][1149/1563]	Loss   0.57	Acc  81.43	Time/batch 0.057
Epoch: [9][1199/1563]	Loss   0.57	Acc  81.40	Time/batch 0.057
Epoch: [9][1249/1563]	Loss   0.57	Acc  81.39	Time/batch 0.057
Epoch: [9][1299/1563]	Loss   0.57	Acc  81.36	Time/batch 0.057
Epoch: [9][1349/1563]	Loss   0.57	Acc  81.39	Time/batch 0.057
Epoch: [

Epoch: [13][ 749/1563]	Loss   0.49	Acc  83.95	Time/batch 0.057
Epoch: [13][ 799/1563]	Loss   0.49	Acc  84.00	Time/batch 0.057
Epoch: [13][ 849/1563]	Loss   0.49	Acc  84.02	Time/batch 0.057
Epoch: [13][ 899/1563]	Loss   0.49	Acc  83.96	Time/batch 0.057
Epoch: [13][ 949/1563]	Loss   0.49	Acc  83.97	Time/batch 0.057
Epoch: [13][ 999/1563]	Loss   0.49	Acc  83.85	Time/batch 0.057
Epoch: [13][1049/1563]	Loss   0.49	Acc  83.79	Time/batch 0.057
Epoch: [13][1099/1563]	Loss   0.49	Acc  83.75	Time/batch 0.057
Epoch: [13][1149/1563]	Loss   0.49	Acc  83.72	Time/batch 0.057
Epoch: [13][1199/1563]	Loss   0.49	Acc  83.77	Time/batch 0.057
Epoch: [13][1249/1563]	Loss   0.49	Acc  83.78	Time/batch 0.057
Epoch: [13][1299/1563]	Loss   0.49	Acc  83.82	Time/batch 0.057
Epoch: [13][1349/1563]	Loss   0.49	Acc  83.81	Time/batch 0.057
Epoch: [13][1399/1563]	Loss   0.49	Acc  83.82	Time/batch 0.057
Epoch: [13][1449/1563]	Loss   0.49	Acc  83.85	Time/batch 0.057
Epoch: [13][1499/1563]	Loss   0.49	Acc  83.84	Time/batc

Epoch: [17][ 999/1563]	Loss   0.45	Acc  84.96	Time/batch 0.057
Epoch: [17][1049/1563]	Loss   0.44	Acc  85.00	Time/batch 0.057
Epoch: [17][1099/1563]	Loss   0.44	Acc  85.02	Time/batch 0.057
Epoch: [17][1149/1563]	Loss   0.44	Acc  85.02	Time/batch 0.057
Epoch: [17][1199/1563]	Loss   0.44	Acc  85.02	Time/batch 0.057
Epoch: [17][1249/1563]	Loss   0.45	Acc  84.97	Time/batch 0.057
Epoch: [17][1299/1563]	Loss   0.44	Acc  85.04	Time/batch 0.057
Epoch: [17][1349/1563]	Loss   0.44	Acc  85.02	Time/batch 0.057
Epoch: [17][1399/1563]	Loss   0.45	Acc  84.99	Time/batch 0.057
Epoch: [17][1449/1563]	Loss   0.44	Acc  84.98	Time/batch 0.057
Epoch: [17][1499/1563]	Loss   0.44	Acc  84.99	Time/batch 0.057
Epoch: [17][1549/1563]	Loss   0.44	Acc  85.03	Time/batch 0.057
current learning rate = 0.025
Epoch: [18][  49/1563]	Loss   0.44	Acc  85.81	Time/batch 0.057
Epoch: [18][  99/1563]	Loss   0.43	Acc  86.16	Time/batch 0.057
Epoch: [18][ 149/1563]	Loss   0.42	Acc  86.42	Time/batch 0.057
Epoch: [18][ 199/1563]	Lo

...

Accuracy of the network on the 10000 test images: 91.0 %
Sparsity of the update phase: 70.0 %
current learning rate = 0.00025000000000000006
Epoch: [181][  49/1563]	Loss   0.08	Acc  97.25	Time/batch 0.058
Epoch: [181][  99/1563]	Loss   0.08	Acc  97.28	Time/batch 0.057
Epoch: [181][ 149/1563]	Loss   0.08	Acc  97.23	Time/batch 0.057
Epoch: [181][ 199/1563]	Loss   0.07	Acc  97.31	Time/batch 0.057
Epoch: [181][ 249/1563]	Loss   0.07	Acc  97.45	Time/batch 0.057
Epoch: [181][ 299/1563]	Loss   0.07	Acc  97.54	Time/batch 0.057
Epoch: [181][ 349/1563]	Loss   0.08	Acc  97.46	Time/batch 0.057
Epoch: [181][ 399/1563]	Loss   0.08	Acc  97.41	Time/batch 0.057
Epoch: [181][ 449/1563]	Loss   0.08	Acc  97.35	Time/batch 0.057
Epoch: [181][ 499/1563]	Loss   0.07	Acc  97.40	Time/batch 0.057
Epoch: [181][ 549/1563]	Loss   0.07	Acc  97.38	Time/batch 0.057
Epoch: [181][ 599/1563]	Loss   0.07	Acc  97.44	Time/batch 0.057
Epoch: [181][ 649/1563]	Loss   0.07	Acc  97.47	Time/batch 0.057
Epoch: [181][ 699/1563]	Los

current learning rate = 0.00025000000000000006
Epoch: [185][  49/1563]	Loss   0.08	Acc  97.62	Time/batch 0.058
Epoch: [185][  99/1563]	Loss   0.07	Acc  97.69	Time/batch 0.058
Epoch: [185][ 149/1563]	Loss   0.07	Acc  97.52	Time/batch 0.058
Epoch: [185][ 199/1563]	Loss   0.07	Acc  97.55	Time/batch 0.057
Epoch: [185][ 249/1563]	Loss   0.07	Acc  97.40	Time/batch 0.057
Epoch: [185][ 299/1563]	Loss   0.08	Acc  97.32	Time/batch 0.057
Epoch: [185][ 349/1563]	Loss   0.07	Acc  97.37	Time/batch 0.057
Epoch: [185][ 399/1563]	Loss   0.07	Acc  97.42	Time/batch 0.057
Epoch: [185][ 449/1563]	Loss   0.07	Acc  97.37	Time/batch 0.057
Epoch: [185][ 499/1563]	Loss   0.07	Acc  97.34	Time/batch 0.057
Epoch: [185][ 549/1563]	Loss   0.07	Acc  97.41	Time/batch 0.057
Epoch: [185][ 599/1563]	Loss   0.07	Acc  97.40	Time/batch 0.057
Epoch: [185][ 649/1563]	Loss   0.07	Acc  97.41	Time/batch 0.057
Epoch: [185][ 699/1563]	Loss   0.07	Acc  97.36	Time/batch 0.057
Epoch: [185][ 749/1563]	Loss   0.07	Acc  97.37	Time/batch

Epoch: [189][  99/1563]	Loss   0.08	Acc  97.16	Time/batch 0.058
Epoch: [189][ 149/1563]	Loss   0.08	Acc  97.02	Time/batch 0.058
Epoch: [189][ 199/1563]	Loss   0.08	Acc  97.08	Time/batch 0.058
Epoch: [189][ 249/1563]	Loss   0.08	Acc  97.17	Time/batch 0.057
Epoch: [189][ 299/1563]	Loss   0.08	Acc  97.33	Time/batch 0.057
Epoch: [189][ 349/1563]	Loss   0.08	Acc  97.35	Time/batch 0.057
Epoch: [189][ 399/1563]	Loss   0.08	Acc  97.39	Time/batch 0.057
Epoch: [189][ 449/1563]	Loss   0.08	Acc  97.43	Time/batch 0.057
Epoch: [189][ 499/1563]	Loss   0.07	Acc  97.47	Time/batch 0.057
Epoch: [189][ 549/1563]	Loss   0.07	Acc  97.46	Time/batch 0.057
Epoch: [189][ 599/1563]	Loss   0.07	Acc  97.47	Time/batch 0.057
Epoch: [189][ 649/1563]	Loss   0.07	Acc  97.48	Time/batch 0.057
Epoch: [189][ 699/1563]	Loss   0.07	Acc  97.47	Time/batch 0.057
Epoch: [189][ 749/1563]	Loss   0.08	Acc  97.45	Time/batch 0.057
Epoch: [189][ 799/1563]	Loss   0.08	Acc  97.45	Time/batch 0.057
Epoch: [189][ 849/1563]	Loss   0.07	Acc 

Epoch: [193][  99/1563]	Loss   0.07	Acc  97.72	Time/batch 0.058
Epoch: [193][ 149/1563]	Loss   0.08	Acc  97.58	Time/batch 0.058
Epoch: [193][ 199/1563]	Loss   0.07	Acc  97.61	Time/batch 0.058
Epoch: [193][ 249/1563]	Loss   0.07	Acc  97.62	Time/batch 0.058
Epoch: [193][ 299/1563]	Loss   0.08	Acc  97.56	Time/batch 0.057
Epoch: [193][ 349/1563]	Loss   0.07	Acc  97.62	Time/batch 0.057
Epoch: [193][ 399/1563]	Loss   0.07	Acc  97.56	Time/batch 0.057
Epoch: [193][ 449/1563]	Loss   0.07	Acc  97.56	Time/batch 0.057
Epoch: [193][ 499/1563]	Loss   0.07	Acc  97.57	Time/batch 0.057
Epoch: [193][ 549/1563]	Loss   0.07	Acc  97.55	Time/batch 0.057
Epoch: [193][ 599/1563]	Loss   0.07	Acc  97.61	Time/batch 0.057
Epoch: [193][ 649/1563]	Loss   0.07	Acc  97.59	Time/batch 0.057
Epoch: [193][ 699/1563]	Loss   0.07	Acc  97.60	Time/batch 0.057
Epoch: [193][ 749/1563]	Loss   0.07	Acc  97.62	Time/batch 0.057
Epoch: [193][ 799/1563]	Loss   0.07	Acc  97.60	Time/batch 0.057
Epoch: [193][ 849/1563]	Loss   0.07	Acc 

Epoch: [197][ 199/1563]	Loss   0.07	Acc  97.50	Time/batch 0.059
Epoch: [197][ 249/1563]	Loss   0.08	Acc  97.36	Time/batch 0.058
Epoch: [197][ 299/1563]	Loss   0.07	Acc  97.45	Time/batch 0.058
Epoch: [197][ 349/1563]	Loss   0.07	Acc  97.43	Time/batch 0.058
Epoch: [197][ 399/1563]	Loss   0.08	Acc  97.37	Time/batch 0.058
Epoch: [197][ 449/1563]	Loss   0.08	Acc  97.35	Time/batch 0.058
Epoch: [197][ 499/1563]	Loss   0.08	Acc  97.33	Time/batch 0.058
Epoch: [197][ 549/1563]	Loss   0.08	Acc  97.34	Time/batch 0.058
Epoch: [197][ 599/1563]	Loss   0.08	Acc  97.33	Time/batch 0.058
Epoch: [197][ 649/1563]	Loss   0.07	Acc  97.38	Time/batch 0.058
Epoch: [197][ 699/1563]	Loss   0.08	Acc  97.34	Time/batch 0.058
Epoch: [197][ 749/1563]	Loss   0.08	Acc  97.37	Time/batch 0.058
Epoch: [197][ 799/1563]	Loss   0.08	Acc  97.36	Time/batch 0.057
Epoch: [197][ 849/1563]	Loss   0.08	Acc  97.35	Time/batch 0.057
Epoch: [197][ 899/1563]	Loss   0.08	Acc  97.35	Time/batch 0.057
Epoch: [197][ 949/1563]	Loss   0.08	Acc 